In [1]:
import glob
import math
import sys
import time
from pathlib import Path
from typing import Optional, Tuple, Union
import math
import lightning as L
import torch
from lightning.fabric.strategies import FSDPStrategy, XLAStrategy
from torch.utils.data import DataLoader
from functools import partial
# support running without installing as a package
wd = Path("/home/aiops/wangsd/TinyLlama_final/TinyLlama/pretrain/tinyllama_v5.py").parent.parent.resolve()
# wd = Path(__file__).parent.parent.resolve()
sys.path.append(str(wd))
# from apex.optimizers import FusedAdam #torch optimizer has a cuda backend, which is faster actually
from lit_gpt.model import GPT, Block, Config, CausalSelfAttention
from lit_gpt.packed_dataset import CombinedDataset, PackedDataset
from lit_gpt.speed_monitor import SpeedMonitorFabric as Monitor
from lit_gpt.speed_monitor import estimate_flops, measure_flops
from lit_gpt.utils import chunked_cross_entropy, get_default_supported_precision, num_parameters, step_csv_logger, lazy_load
from pytorch_lightning.loggers import WandbLogger
from lit_gpt import FusedCrossEntropyLoss
import random

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


/home/aiops/wangsd/miniforge3/envs/TL/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/aiops/wangsd/miniforge3/envs/TL/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cpu.so
/home/aiops/wangsd/miniforge3/envs/TL/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32
CUDA SETUP: Loading binary /home/aiops/wangsd/miniforge3/envs/TL/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cpu.so...


/home/aiops/wangsd/miniforge3/envs/TL/lib/python3.11/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


In [2]:
from tinyllama_v5 import create_dataloaders

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: sanderwang. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/aiops/wangsd/.netrc


In [3]:

# model_name = "tiny_LLaMA_1b"
model_name = "tiny_Mamba_120m_p"
# name = "tinyllama_1b"
name = "tiny_Mamba_120m_p_slow"
out_dir = Path("out") / name
version = 5

# Hyperparameters
num_of_devices = 2
global_batch_size = 256
learning_rate = 6e-4
# micro_batch_size = 8
micro_batch_size = 1
max_step = 4800
warmup_steps = 48
log_step_interval = 4
eval_iters = 100
save_step_interval = 400
eval_step_interval = 400


weight_decay = 1e-1
# weight_decay = 0.000
beta1 = 0.9
beta2 = 0.95
grad_clip = 1.0
decay_lr = True
min_lr = 4e-5

batch_size = global_batch_size // num_of_devices
gradient_accumulation_steps = batch_size // micro_batch_size
assert gradient_accumulation_steps > 0
warmup_iters = warmup_steps * gradient_accumulation_steps




max_iters = max_step * gradient_accumulation_steps
lr_decay_iters = max_iters
log_iter_interval = log_step_interval * gradient_accumulation_steps


# Treat all dataset equally by their size. If you want to use a different weight for a dataset, add it to the list with the weight.
# train_data_config = [
#     ("train_slim", 0.693584),
#     ("train_star", 0.306416),
# ]

# val_data_config = [
#     ("validation", 1.0),
# ]


train_data_config = [
    ("train_ind", 1.0),
]

val_data_config = [
    ("train_ind", 1.0),
]

hparams = {k: v for k, v in locals().items() if isinstance(v, (int, float, str)) and not k.startswith("_")}

In [4]:
config = Config.from_name(model_name)
devices = 1
strategy = "auto"
precision = "32-true"

fabric = L.Fabric(devices=devices, strategy=strategy, precision=precision, loggers=[])
fabric.print(hparams)

train_data_dir = Path("/home/aiops/wangsd/TinyLlama/data/the_pile_deduplicated_EleutherAI_split_combined")
val_data_dir = Path("/home/aiops/wangsd/TinyLlama/data/the_pile_deduplicated_EleutherAI_split_combined")

train_dataloader, val_dataloader = create_dataloaders(
        batch_size=micro_batch_size,
        block_size=config.block_size,
        fabric=fabric,
        train_data_dir=train_data_dir,
        val_data_dir=val_data_dir,
        seed=3407,
    )

{'model_name': 'tiny_Mamba_120m_p', 'name': 'tiny_Mamba_120m_p_slow', 'version': 5, 'num_of_devices': 2, 'global_batch_size': 256, 'learning_rate': 0.0006, 'micro_batch_size': 1, 'max_step': 4800, 'warmup_steps': 48, 'log_step_interval': 4, 'eval_iters': 100, 'save_step_interval': 400, 'eval_step_interval': 400, 'weight_decay': 0.1, 'beta1': 0.9, 'beta2': 0.95, 'grad_clip': 1.0, 'decay_lr': True, 'min_lr': 4e-05, 'batch_size': 128, 'gradient_accumulation_steps': 128, 'warmup_iters': 6144, 'max_iters': 614400, 'lr_decay_iters': 614400, 'log_iter_interval': 512}


In [5]:
for index, train_data in enumerate(train_dataloader):
    if index >= 3:
        break

    print(train_data.shape)
    print(train_data)

OSError: [Errno 19] No such device

In [ ]:
train_data[:,:-1]

tensor([[  355, 29432, 12210,  ...,  4322,   313,    51]])

In [ ]:
train_data[:,1:]

tensor([[29432, 12210,  8088,  ...,   313,    51,  4542]])